In [2]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
import sys
from PIL import Image
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data


In [3]:
def convert_color_mapping(labels,color_mapping):
    mapping_from_color2trainid = {}
    for item in color_mapping["labels"]:
        readable = item["readable"].lower()
        if readable in labels:
            color = item["color"]
            mapping_from_color2trainid[tuple(color)]=labels[readable]
    return mapping_from_color2trainid

In [4]:
def rgb_to_object_id(rgb_value, mapping_from_color2trainid):
    if tuple(rgb_value) in mapping_from_color2trainid:
        return mapping_from_color2trainid[tuple(rgb_value)]
    else:
        return 255

In [5]:
def convert_masks(input_folder, output_folder, mapping_from_color2trainid):
    os.makedirs(output_folder, exist_ok=True)

    for filename in tqdm(os.listdir(input_folder), desc="Processing files", unit="file"):
        if filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            mask = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Convert RGB values to object IDs
            mask_object_ids = np.zeros_like(mask[:, :, 0], dtype=np.uint8)

            for rgb_value, obj_id in mapping_from_color2trainid.items():
                rgb_value = list(rgb_value)
                mask_object_ids[np.all(mask == rgb_value, axis=-1)] = obj_id

            # Save the grayscale image
            output_path = os.path.join(output_folder, filename)
            img = Image.fromarray(mask_object_ids)
            img = img.resize((256,256))
            img.save(output_path)

In [13]:
if __name__ == "__main__":
    labels = {"unlabeled": 0,
              "road": 1,
              "sidewalk":2 ,
              "traffic light": 3,
              "traffic sign": 4,
              "person": 5,
              "car": 6,
              "bicycle":7}

    json_path = "/kaggle/input/mapillary/Mapillary-Vistas-1000-sidewalks/config.json"
    color_mapping = load_json(json_path)

    mapping_from_color2trainid = convert_color_mapping(labels,color_mapping)


#     training_folder ="/kaggle/input/mapillary/Mapillary-Vistas-1000-sidewalks/training/labels"
#     training_output_folder ="mapillary_converted/training/labels"
#     convert_masks(training_folder, training_output_folder, mapping_from_color2trainid)

#     validation_folder = "/kaggle/input/mapillary/Mapillary-Vistas-1000-sidewalks/validation/labels"
#     validation_output_folder = "mapillary_converted/validation/labels"
#     convert_masks(validation_folder, validation_output_folder, mapping_from_color2trainid)

    testing_folder = "/kaggle/input/mapillary/Mapillary-Vistas-1000-sidewalks/testing/labels"
    testing_output_folder = "/mapillary_converted/testing/labels"
    convert_masks(testing_folder, testing_output_folder, mapping_from_color2trainid)

Processing files: 100%|██████████| 200/200 [08:07<00:00,  2.44s/file]


In [ ]:
/kaggle/working/mapillary_converted